In [ ]:
#!pip install langchain

In [ ]:
from langchain import SQLDatabase, SQLDatabaseChain
from langchain.llms import VertexAI
# from langchain.sql_database import SQLDatabase
# from langchain import SQLDatabaseChain

In [20]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql


PROJECT_ID="jared-playground"
REGION="us-central1"
INSTANCE_NAME="gen-ai-text-to-mysql"

# initialize parameters
INSTANCE_CONNECTION_NAME = f"{PROJECT_ID}:{REGION}:{INSTANCE_NAME}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "root"
DB_NAME = "classicmodels"

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
# pool = sqlalchemy.create_engine(
#     "mysql+pymysql://",
#     creator=getconn,
# )

db = SQLDatabase.from_uri(
    "mysql+pymysql://",
    engine_args = {'creator': getconn},
)

Your instance connection name is: jared-playground:us-central1:gen-ai-text-to-mysql


In [21]:
llm = VertexAI(model_name="code-bison@001", temperature=0, verbose=True)

In [22]:
db_chain = SQLDatabaseChain.from_llm(llm, db)

In [23]:
print(db.table_info)


CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)ENGINE=InnoDB DEFAULT CHARSET=latin1

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmitt	Carine 	40.32.2555	54, rue Royale	None	Nantes	None	44000	France	1370	21000.00
112	Signal Gift Stores	King	Jean	7025551838	8489 Strong St

In [24]:
db_chain.run("How many orders were placed?")

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Question: How many orders were placed?\nSQLQuery: SELECT COUNT(*) FROM orders' at line 1")
[SQL: Question: How many orders were placed?
SQLQuery: SELECT COUNT(*) FROM orders]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Be sure to put a semi-colon after query.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone asks for the table foobar, they really mean the employee table.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)
PROMPT.format(input="How many orders were placed in the year 2005?", table_info="orders", dialect="MySQL 8")

In [ ]:
# db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("How many orders were placed?")

In [10]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [12]:
from langchain.agents import create_sql_agent
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [14]:
agent_executor.run("Describe the orders table")



> Entering new AgentExecutor chain...
Question: Describe the orders table
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables. 
Action: sql_db_list_tables
Action Input: 
Observation: customers, employees, offices, orderdetails, orders, payments, productlines, products
Thought:Question: Describe the orders table
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables. 
Action: sql_tb_schema
Action Input: orders
Observation: sql_tb_schema is not a valid tool, try another one.
Thought:Question: Describe the orders table
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables. 
Action: sql_tb_schema
Action Input: orders
Observation: sql_tb_schema is not a valid tool, try another one.
Thought:

OutputParserException: Could not parse LLM output: `Question: Describe the orders table
Error: Unknown tool 'sql_tb_schema'. Please try one of: ['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker'].`

In [ ]:
import os

import sqlalchemy


pool = sqlalchemy.create_engine(
    # Equivalent URL:
    # "mysql+pymysql://root@/classicmodels?unix_socket=/cloudsql/jared-playground:gen-ai-text-to-mysql"
    sqlalchemy.engine.url.URL.create(
        drivername="mysql+pymysql",
        username='root',
        database='classicmodels',
        query={"unix_socket": '/cloudsql/jared-playground:us-central1:gen-ai-text-to-mysql'}
    ),
    # ...
)

print(pool)

In [ ]:
URL = sqlalchemy.engine.url.URL.create(
        drivername="mysql+pymysql",
        username='root',
        database='classicmodels',
        host='34.122.217.224',
        # port='3306',
)
print(URL)

In [ ]:
DB_USER = "root"
DB_NAME = "classicmodels"
DB_ADDRESS = "34.122.217.224"
SOCKET = "mysql+pymysql://root@/classicmodels?unix_socket=/cloudsql/jared-playground:gen-ai-text-to-mysql"

db = SQLDatabase.from_uri(URL)
# db = SQLDatabase.from_uri(f"mysql+pymysql://{DB_USER}:root@{DB_ADDRESS}/{DB_NAME}")
# llm = VertexAI(model_name="code-bison")

In [ ]:
DB_USER = "root"
DB_NAME = "classicmodels"
DB_ADDRESS = "34.122.217.224"
SOCKET = "mysql+pymysql://root:root@/classicmodels?unix_socket=/cloudsql/jared-playground:gen-ai-text-to-mysql"

db = SQLDatabase.from_uri(SOCKET)